<a href="https://colab.research.google.com/github/chayan141/nlp-playlist/blob/main/NLP_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Learning Rate Scheduling

Learning rate scheduling is a technique used in training machine learning models, particularly neural networks, to dynamically adjust the learning rate during training. The learning rate is a hyperparameter that controls how much the model's parameters (e.g., weights) are updated in response to the gradient of the loss function during optimization.

Early Training: A larger learning rate can help the model explore the parameter space quickly and escape suboptimal local minima.

Later training: A smaller learning rate allows the model to make finer adjustment and converging more precisely.

Methods:

1. Step decay: The learning rate is reduced by a fixed factor after a specified number of epochs or iterations.

  Example: learning rate is reduced by half for every 10 epoch.

2. Exponential Decay: Learning rate decreses exponentially over time. for smooth convergence.

3. Polynomial Decay: The learning rate decreases according to a polynomial function, often used in linear or power-law decay.

4. Cosine Annealing: The learning rate follows a cosine function, decreasing gradually at first, then rapidly, and slowing down again.

4. ReduceLRon Plateau: The learning rate is reduced when a monitored metric (e.g., validation loss) stops improving (plateaus).

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import ReduceLROnPlateau, CSVLogger
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd